# **CS 245 Machine Learning Course Project: Micro-Scale Road Congestion Prediction in Lahore**

**Group Members:** [Faryal Khan, 465125], [Wania Mateen, ID], [Haleema Imran, ID]  
**Theme:** T1: Urban Intelligence & City Futures  
**Problem:** Predict 10-15 min horizon congestion hotspots on Lahore roads using accidents, mobility, weather, and OSM data.  
**Stakeholders:** Lahore Traffic Police, commuters.  

This notebook implements the end-to-end pipeline: data curation, ML models, evaluation, and PoC prep. Run top-to-bottom after populating `data/raw/`.  

## **Setup**

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import joblib
import warnings
import os
import pandas as pd
warnings.filterwarnings("ignore")

# Paths (team: adjust if needed)
raw_path = "data/raw/"
processed_path = "data/processed/"
models_path = "models/"
os.makedirs(processed_path, exist_ok=True)
os.makedirs(models_path, exist_ok=True)

print("Setup complete! Paths ready.")

Setup complete! Paths ready.


In [6]:
print("=== RTA Accidents Dataset Exploration ===")
rta_path = "data/raw/RTA Data 2020 to July 2023.xlsx"
rta = pd.read_excel(rta_path)
print(f"Shape: {rta.shape} (rows, cols)")
print(f"Columns: {rta.columns.tolist()}")
print(f"Data types:\n{rta.dtypes}")
print(f"Missing values per column:\n{rta.isnull().sum()}")
print("\nFirst 3 rows:\n", rta.head(3).to_string())
print("\nDate-like columns sample (first 5 unique dates):\n")
date_cols = [col for col in rta.columns if 'date' in col.lower() or 'time' in col.lower()]
for col in date_cols[:3]:  # Top 3 potential date/time cols
    print(f"{col}: {rta[col].dropna().unique()[:5]}")
print("\nLat/Lon sample (first 5):\n", rta[[col for col in rta.columns if 'lat' in col.lower() or 'lon' in col.lower()]].head())

=== RTA Accidents Dataset Exploration ===
Shape: (46189, 25) (rows, cols)
Columns: ['EcYear', 'EcNumber', 'CallTime', 'EmergencyArea', 'TotalPatientsInEmergency', 'Gender', 'Age', 'HospitalName', 'Reason', 'responsetime', 'EducationTitle', 'InjuryType', 'Cause', 'PatientStatus', 'BicycleInvovled', 'BikesInvolved', 'BusesInvolved', 'CarsInvolved', 'CartInvovled', 'RickshawsInvolved', 'TractorInvovled', 'TrainsInvovled', 'TrucksInvolved', 'VansInvolved', 'OthersInvolved']
Data types:
EcYear                              object
EcNumber                            object
CallTime                    datetime64[ns]
EmergencyArea                       object
TotalPatientsInEmergency            object
Gender                              object
Age                                float64
HospitalName                        object
Reason                              object
responsetime                       float64
EducationTitle                      object
InjuryType                          obje

In [4]:
print("=== Loading PK Mobility Files ===")
files = [
    "data/raw/2020_PK_Region_Mobility_Report.xlsx",
    "data/raw/2021_PK_Region_Mobility_Report.xlsx",
    "data/raw/2022_PK_Region_Mobility_Report.xlsx"
]

dfs = []
for file in files:
    df = pd.read_excel(file)
    df['datetime'] = pd.to_datetime(df['date'])
    dfs.append(df)
mob_pk = pd.concat(dfs, ignore_index=True)
print(f"Concat shape: {mob_pk.shape}")
print("Columns:", mob_pk.columns.tolist())
print("Unique sub_region_1:", mob_pk['sub_region_1'].unique())
print("Unique sub_region_2:", mob_pk['sub_region_2'].unique()[:10])  # Top 10
print("\nSample:\n", mob_pk.head(3)[['sub_region_1', 'sub_region_2', 'date', 'workplaces_percent_change_from_baseline']].to_string())

=== Loading PK Mobility Files ===
Concat shape: (20404, 16)
Columns: ['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date', 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline', 'residential_percent_change_from_baseline', 'datetime']
Unique sub_region_1: [nan 'Azad Jammu and Kashmir' 'Balochistan'
 'Federally Administered Tribal Area' 'Gilgit-Baltistan'
 'Islamabad Capital Territory' 'Khyber Pakhtunkhwa' 'Punjab' 'Sindh']
Unique sub_region_2: [nan]

Sample:
   sub_region_1  sub_region_2       date  workplaces_percent_change_from_baseline
0          NaN           NaN 2020-02-15                                      2.0
1          NaN           NaN 2020-02-16                                      2.0
2          NaN     

In [9]:
print("=== Weather Datasets Exploration ===")
w1_path = "data/raw/pakistan_weather_2000_2024.csv"
w2_path = "data/raw/pakistan_weather_data-Sep2024-Oct2025.csv"
w1 = pd.read_csv(w1_path)
w2 = pd.read_csv(w2_path)
weather = pd.concat([w1, w2], ignore_index=True)
print(f"Combined shape: {weather.shape}")
print(f"Columns: {weather.columns.tolist()}")
print(f"Data types:\n{weather.dtypes}")
print(f"Missing values per column:\n{weather.isnull().sum()}")
print("\nFirst 3 rows:\n", weather.head(3).to_string())
print("\nLahore sample (first 5 rows):\n")
lahore_sample = weather[weather['city'].str.contains('Lahore', na=False, case=False)].head()
print(lahore_sample.to_string())
print(f"\nUnique cities: {weather['city'].unique()[:10]}")
print(f"Date range: {weather['date'].min()} to {weather['date'].max()}")

=== Weather Datasets Exploration ===
Combined shape: (34245, 27)
Columns: ['date', 'year', 'month', 'day', 'dayofweek', 'is_weekend', 'season', 'city', 'region', 'latitude', 'longitude', 'elevation', 'tmin', 'tmax', 'tavg', 'prcp', 'wspd', 'humidity', 'pressure', 'dew_point', 'cloud_cover', 'visibility', 'temp_range', 'is_hot_day', 'is_cold_day', 'rainfall_intensity', 'wind_category']
Data types:
date                   object
year                    int64
month                   int64
day                     int64
dayofweek               int64
is_weekend              int64
season                 object
city                   object
region                 object
latitude              float64
longitude             float64
elevation             float64
tmin                  float64
tmax                  float64
tavg                  float64
prcp                  float64
wspd                  float64
humidity              float64
pressure              float64
dew_point             float64


In [2]:
print("=== Roads Dataset Exploration (From Local PBF - Offline) ===")

import osmium
from shapely.geometry import LineString
import geopandas as gpd
import numpy as np

# Lahore bbox for filtering (min_lat, min_lon, max_lat, max_lon)
lahore_bounds = (31.4, 74.1, 31.7, 74.5)

# Handler to extract nodes and ways (roads) in bbox
class LahoreHandler(osmium.SimpleHandler):
    def __init__(self):
        super().__init__()
        self.nodes = {}  # Store node coords
        self.ways = []  # Store roads

    def node(self, n):
        lat, lon = n.location.lat, n.location.lon
        if lahore_bounds[0] <= lat <= lahore_bounds[2] and lahore_bounds[1] <= lon <= lahore_bounds[3]:
            self.nodes[n.id] = (lat, lon)

    def way(self, w):
        if 'highway' in w.tags:  # Only roads
            coords = []
            for nd in w.nodes:
                if nd in self.nodes:
                    coords.append(self.nodes[nd])
            if len(coords) >= 2:  # Valid line
                geometry = LineString(coords)
                # Approx length in km (haversine simple for equatorial approx; Lahore is ~31°N)
                length = sum(np.sqrt((coords[i][1] - coords[i+1][1])**2 + ((coords[i][0] - coords[i+1][0]) * np.cos(np.radians(coords[i][0])) )**2 ) * 111 for i in range(len(coords)-1)) / 1000
                self.ways.append({
                    'highway': w.tags.get('highway', 'unknown'),
                    'name': w.tags.get('name', 'unnamed'),
                    'geometry': geometry,
                    'length': length
                })

# Process PBF
handler = LahoreHandler()
handler.apply_file("data/raw/pakistan-251119.osm.pbf")
print(f"Extracted {len(handler.ways)} road segments from PBF.")

if len(handler.ways) == 0:
    print("No roads found—widen bbox or check PBF.")
else:
    # Create GeoDataFrame with explicit geometry column
    roads_gdf = gpd.GeoDataFrame(handler.ways, geometry='geometry', crs="EPSG:4326")
    roads_gdf["segment_id"] = range(len(roads_gdf))

    print(f"Shape: {roads_gdf.shape} (road segments)")
    print(f"Columns: {roads_gdf.columns.tolist()}")
    print(f"Data types:\n{roads_gdf.dtypes}")
    print(f"Missing values per column:\n{roads_gdf.isnull().sum()}")
    print("\nFirst 3 segments:\n", roads_gdf.head(3)[['highway', 'name', 'length', 'geometry']].to_string())
    print(f"Road types (top 5): \n{roads_gdf['highway'].value_counts().head()}")
    print(f"Average length: {roads_gdf['length'].mean():.2f} km")
    print(f"Total road length: {roads_gdf['length'].sum():.1f} km")

=== Roads Dataset Exploration (From Local PBF - Offline) ===
Extracted 0 road segments from PBF.
No roads found—widen bbox or check PBF.


In [6]:
import geopandas as gpd
from shapely import from_wkt  # Use shapely directly for single WKT

print("SHX restore enabled.")

print("=== Roads from Shapefile ===")
roads_path = "data/raw/hotosm_pak_roads_lines_shp.shp"  # Confirm with os.listdir if needed
roads_gdf = gpd.read_file(roads_path)

# Filter Lahore (bbox polygon using shapely for single WKT)
lahore_wkt = 'POLYGON((74.1 31.4, 74.5 31.4, 74.5 31.7, 74.1 31.7, 74.1 31.4))'
lahore_poly = from_wkt(lahore_wkt)
roads_gdf = roads_gdf[roads_gdf.intersects(lahore_poly)]
roads_gdf["segment_id"] = range(len(roads_gdf))
if 'length' not in roads_gdf.columns:
    roads_gdf["length"] = roads_gdf.geometry.length / 1000  # km
print(f"Shape: {roads_gdf.shape}")
print(f"Columns: {roads_gdf.columns.tolist()}")
print(roads_gdf.head(3))
print(f"Road types (top 5): \n{roads_gdf['highway'].value_counts().head() if 'highway' in roads_gdf.columns else 'No highway col'}")
print(f"Average length: {roads_gdf['length'].mean():.2f} km")

SHX restore enabled.
=== Roads from Shapefile ===
Shape: (79354, 3)
Columns: ['geometry', 'segment_id', 'length']
                                              geometry  segment_id  \
227  LINESTRING (74.29559 31.47191, 74.29662 31.47157)           0   
228  LINESTRING (74.29684 31.47189, 74.29662 31.47157)           1   
229  LINESTRING (74.29924 31.46942, 74.29917 31.469...           2   

           length  
227  1.083790e-06  
228  3.834327e-07  
229  3.594147e-07  
Road types (top 5): 
No highway col
Average length: 0.00 km


In [9]:
import geopandas as gpd
from shapely import from_wkt
import os
os.environ['SHAPE_RESTORE_SHX'] = 'YES'  # SHX restore

print("=== Roads from Shapefile (Warning Fixed) ===")
roads_path = "data/raw/hotosm_pak_roads_lines_shp.shp"  # Confirm with os.listdir if needed
roads_gdf = gpd.read_file(roads_path)

# Set original CRS (assume WGS84 for OSM shapefile)
roads_gdf = roads_gdf.set_crs("EPSG:4326", allow_override=True)

# Project to UTM for metric lengths (Lahore zone 43N)
roads_gdf_projected = roads_gdf.to_crs("EPSG:32643")
roads_gdf_projected["length"] = roads_gdf_projected.geometry.length / 1000  # Now in km
roads_gdf = roads_gdf_projected.to_crs("EPSG:4326")  # Back to WGS84

# Filter Lahore (bbox polygon) and copy to avoid warning
lahore_wkt = 'POLYGON((74.1 31.4, 74.5 31.4, 74.5 31.7, 74.1 31.7, 74.1 31.4))'
lahore_poly = from_wkt(lahore_wkt)
lahore_roads = roads_gdf[roads_gdf.intersects(lahore_poly)].copy()  # .copy() fixes warning

# Safe assignments with .loc
lahore_roads.loc[:, "segment_id"] = range(len(lahore_roads))
if 'highway' not in lahore_roads.columns:
    lahore_roads.loc[:, "highway"] = "unknown"
if 'name' not in lahore_roads.columns:
    lahore_roads.loc[:, "name"] = "unnamed"

print(f"Full shape: {roads_gdf.shape}, Lahore filter shape: {lahore_roads.shape}")
print(f"Columns: {lahore_roads.columns.tolist()}")
print(f"Data types:\n{lahore_roads.dtypes}")
print(f"Missing values per column:\n{lahore_roads.isnull().sum()}")
print("\nFirst 3 segments:\n", lahore_roads.head(3)[['highway', 'name', 'length', 'geometry']].to_string())
print(f"Road types (top 5): \n{lahore_roads['highway'].value_counts().head()}")
print(f"Average length: {lahore_roads['length'].mean():.2f} km")
print(f"Total road length: {lahore_roads['length'].sum():.1f} km")

=== Roads from Shapefile (Warning Fixed) ===
Full shape: (1077644, 2), Lahore filter shape: (79354, 5)
Columns: ['geometry', 'length', 'segment_id', 'highway', 'name']
Data types:
geometry      geometry
length         float64
segment_id       int64
highway         object
name            object
dtype: object
Missing values per column:
geometry      0
length        0
segment_id    0
highway       0
name          0
dtype: int64

First 3 segments:
      highway     name    length                                                                                                                      geometry
227  unknown  unnamed  0.104746                                                                             LINESTRING (74.29559 31.47191, 74.29662 31.47157)
228  unknown  unnamed  0.040637                                                                             LINESTRING (74.29684 31.47189, 74.29662 31.47157)
229  unknown  unnamed  0.038187  LINESTRING (74.29924 31.46942, 74.29917 31.4

In [12]:
# Clean Accidents
print("=== Cleaning Accidents ===")
rta = pd.read_excel("data/raw/RTA Data 2020 to July 2023.xlsx")

# Basic clean: Drop high-missing cols, handle datetime
rta = rta.dropna(subset=['CallTime'])  # Drop missing time
rta['datetime'] = pd.to_datetime(rta['CallTime'])
rta = rta[rta['datetime'].dt.year.between(2020, 2023)]  # Filter years
rta['incident_count'] = rta['TotalPatientsInEmergency'].fillna(1)  # Proxy (min 1)
rta['severity'] = rta['PatientStatus'# Clean Mobility
print("=== Cleaning Mobility ===")
mob = pd.read_excel("data/raw/Global_Mobility_Report.xlsx")
mob['datetime'] = pd.to_datetime(mob['date'])
mob = mob[mob['datetime'].dt.year.between(2020, 2023)]  # Filter years

# Punjab proxy (no Lahore; use sub_region_1 = 'Punjab')
mob_punjab = mob[
    (mob['country_region'] == 'Pakistan') & 
    (mob['sub_region_1'] == 'Punjab')
][['datetime', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']]

# Resample to 15-min (ffill daily to hourly/15-min for alignment)
mob_punjab = mob_punjab.set_index('datetime').resample('15T').ffill()
mob_punjab['mobility_proxy'] = mob_punjab['workplaces_percent_change_from_baseline'].fillna(0)  # Proxy for traffic demand

print(f"Cleaned Punjab shape: {mob_punjab.shape}")
print(mob_punjab.head(3))
mob_punjab.to_csv("data/processed/cleaned_mobility.csv")].fillna('Unknown')  # Keep for later

# Proxy geo: Simple dict for common areas (Lahore/Rawalpindi; expand if needed)
geo_proxy = {
    'BBH': (31.55, 74.35),  # Benazir Bhutto Hospital approx Lahore
    'Near APS SCHOOL FORT ROAD RWP': (33.60, 73.05),  # Rawalpindi (drop if non-Lahore)
    # Add more from unique EmergencyArea
}
rta['Latitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x, (31.55, 74.35))[0])  # Default Lahore center
rta['Longitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x, (31.55, 74.35))[1])

# Lahore filter (using proxy lat/lon)
lahore_bounds = (31.4, 74.1, 31.7, 74.5)
rta_lahore = rta[
    (rta['Latitude'].between(lahore_bounds[0], lahore_bounds[2])) & 
    (rta['Longitude'].between(lahore_bounds[1], lahore_bounds[3]))
][['datetime', 'Latitude', 'Longitude', 'incident_count', 'severity']]

print(f"Cleaned Lahore shape: {rta_lahore.shape}")
print(rta_lahore.head(3))
rta_lahore.to_csv("data/processed/cleaned_accidents.csv", index=False)

=== Cleaning Accidents ===
Cleaned Lahore shape: (40232, 5)
             datetime  Latitude  Longitude  incident_count          severity
0 2020-12-31 22:41:47     31.55      74.35               1  Alive & unstable
1 2020-12-31 22:25:00     31.55      74.35               1    Alive & stable
2 2020-12-31 21:54:59     31.55      74.35               1  Alive & unstable


In [5]:
# Clean PK Mobility (Based on Your Output)
print("=== Cleaning PK Mobility ===")
files = [
    "data/raw/2020_PK_Region_Mobility_Report.xlsx",
    "data/raw/2021_PK_Region_Mobility_Re# Clean Weather
print("=== Cleaning Weather ===")
w1 = pd.read_csv("data/raw/pakistan_weather_2000_2024.csv")
w2 = pd.read_csv("data/raw/pakistan_weather_data-Sep2024-Oct2025.csv")
w1['datetime'] = pd.to_datetime(w1['date'])
w2['datetime'] = pd.to_datetime(w2['date'])
weather = pd.concat([w1, w2], ignore_index=True)

# Lahore filter & year range
weather_lahore = weather[
    (weather['city'] == 'Lahore') & 
    (weather['datetime'].dt.year.between(2020, 2023))
][['datetime', 'tavg', 'prcp', 'humidity', 'latitude', 'longitude']]

# Resample to 15-min (interpolate for missing)
weather_lahore = weather_lahore.set_index('datetime').resample('15T').interpolate(method='linear')
weather_lahore['temperature'] = weather_lahore['tavg']
weather_lahore['precipitation'] = weather_lahore['prcp']

# Engineer lag for 10-15 min horizon
weather_lahore['precip_lag'] = weather_lahore['precipitation'].shift(4)  # ~1 hour lag (4*15min)

print(f"Cleaned Lahore shape: {weather_lahore.shape}")
print("Head:\n", weather_lahore.head(3))
weather_lahore.to_csv("data/processed/cleaned_weather.csv", index=False)
print("Weather cleaned!")port.xlsx",
    "data/raw/2022_PK_Region_Mobility_Report.xlsx"
]

dfs = []
for file in files:
    df = pd.read_excel(file)
    df['datetime'] = pd.to_datetime(df['date'])
    dfs.append(df)
mob_pk = pd.concat(dfs, ignore_index=True)
mob_pk = mob_pk[mob_pk['datetime'].dt.year.between(2020, 2023)]  # Filter years

# Punjab filter (from your uniques)
mob_punjab = mob_pk[
    (mob_pk['sub_region_1'] == 'Punjab')  # Exact match
][['datetime', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']]

# Resample to 15-min (ffill for alignment with other data)
mob_punjab = mob_punjab.set_index('datetime').resample('15T').ffill()
mob_punjab['mobility_proxy'] = mob_punjab['workplaces_percent_change_from_baseline'].fillna(0)  # Proxy for traffic demand

print(f"Cleaned Punjab shape: {mob_punjab.shape}")
print("Head:\n", mob_punjab.head(3))
mob_punjab.to_csv("data/processed/cleaned_mobility.csv", index=False)
print("Mobility cleaned!")

=== Cleaning PK Mobility ===
Cleaned Punjab shape: (93409, 3)
Head:
                      workplaces_percent_change_from_baseline  \
datetime                                                       
2020-02-15 00:00:00                                      2.0   
2020-02-15 00:15:00                                      2.0   
2020-02-15 00:30:00                                      2.0   

                     transit_stations_percent_change_from_baseline  \
datetime                                                             
2020-02-15 00:00:00                                            4.0   
2020-02-15 00:15:00                                            4.0   
2020-02-15 00:30:00                                            4.0   

                     mobility_proxy  
datetime                             
2020-02-15 00:00:00             2.0  
2020-02-15 00:15:00             2.0  
2020-02-15 00:30:00             2.0  
Mobility cleaned!


In [10]:
# Clean Weather (Fixed Datetime)
print("=== Cleaning Weather ===")
w1 = pd.read_csv("data/raw/pakistan_weather_2000_2024.csv")
w2 = pd.read_csv("data/raw/pakistan_weather_data-Sep2024-Oct2025.csv")
w1['datetime'] = pd.to_datetime(w1['date'], errors='coerce')
w2['datetime'] = pd.to_datetime(w2['date'], errors='coerce')
weather = pd.concat([w1, w2], ignore_index=True)

# Re-apply datetime after concat (ensures consistency)
weather['datetime'] = pd.to_datetime(weather['date'], errors='coerce')
weather = weather.dropna(subset=['datetime'])  # Drop bad dates

# Lahore filter & year range (now .dt works)
weather_lahore = weather[
    (weather['city'] == 'Lahore') & 
    (weather['datetime'].dt.year.between(2020, 2023))
][['datetime', 'tavg', 'prcp', 'humidity', 'latitude', 'longitude']]

# Resample to 15-min (interpolate for missing)
weather_lahore = weather_lahore.set_index('datetime').resample('15T').interpolate(method='linear')
weather_lahore['temperature'] = weather_lahore['tavg']
weather_lahore['precipitation'] = weather_lahore['prcp']

# Engineer lag for 10-15 min horizon
weather_lahore['precip_lag'] = weather_lahore['precipitation'].shift(4)  # ~1 hour lag
# Optional: Ffill lag NaN (propagate first value)
weather_lahore['precip_lag'] = weather_lahore['precip_lag'].fillna(weather_lahore['precipitation'].iloc[0])
print("Lag NaN filled.")
weather_lahore.to_csv("data/processed/cleaned_weather.csv", index=True)  # Re-save

print(f"Cleaned Lahore shape: {weather_lahore.shape}")
print("Head:\n", weather_lahore.head(3))
weather_lahore.to_csv("data/processed/cleaned_weather.csv", index=False)

print("Weather cleaned!")

=== Cleaning Weather ===
Lag NaN filled.
Cleaned Lahore shape: (140161, 8)
Head:
                          tavg  prcp   humidity  latitude  longitude  \
datetime                                                              
2020-01-01 00:00:00  6.200000   0.0  90.000000   31.5497    74.3436   
2020-01-01 00:15:00  6.234375   0.0  89.947917   31.5497    74.3436   
2020-01-01 00:30:00  6.268750   0.0  89.895833   31.5497    74.3436   

                     temperature  precipitation  precip_lag  
datetime                                                     
2020-01-01 00:00:00     6.200000            0.0         0.0  
2020-01-01 00:15:00     6.234375            0.0         0.0  
2020-01-01 00:30:00     6.268750            0.0         0.0  
Weather cleaned!


In [12]:
# Clean Roads
print("=== Cleaning Roads ===")
import geopandas as gpd
from shapely import from_wkt  # Missing import

roads_gdf = gpd.read_file("data/raw/hotosm_pak_roads_lines_shp.shp")
roads_gdf = roads_gdf.set_crs("EPSG:4326", allow_override=True)

# Project for accurate length
roads_projected = roads_gdf.to_crs("EPSG:32643")
roads_projected["length"] = roads_projected.geometry.length / 1000  # km
roads_gdf = roads_projected.to_crs("EPSG:4326")

# Lahore filter (bbox polygon)
lahore_wkt = 'POLYGON((74.1 31.4, 74.5 31.4, 74.5 31.7, 74.1 31.7, 74.1 31.4))'
lahore_poly = from_wkt(lahore_wkt)
lahore_roads = roads_gdf[roads_gdf.intersects(lahore_poly)].copy()

# Add dummies (since no attributes)
lahore_roads["segment_id"] = range(len(lahore_roads))
lahore_roads["highway"] = "unknown"
lahore_roads["name"] = "unnamed"

print(f"Full shape: {roads_gdf.shape}, Lahore shape: {lahore_roads.shape}")
print("Head:\n", lahore_roads.head(3)[['highway', 'name', 'length', 'geometry']])
lahore_roads.to_file("data/processed/cleaned_roads.shp")
print("Roads cleaned!")

=== Cleaning Roads ===
Full shape: (1077644, 2), Lahore shape: (79354, 5)
Head:
      highway     name    length  \
227  unknown  unnamed  0.104746   
228  unknown  unnamed  0.040637   
229  unknown  unnamed  0.038187   

                                              geometry  
227  LINESTRING (74.29559 31.47191, 74.29662 31.47157)  
228  LINESTRING (74.29684 31.47189, 74.29662 31.47157)  
229  LINESTRING (74.29924 31.46942, 74.29917 31.469...  
Roads cleaned!


In [14]:
mob_clean = pd.read_csv("data/processed/cleaned_mobility.csv")
print("Mobility columns:", mob_clean.columns.tolist())
print("Head:\n", mob_clean.head(3))

Mobility columns: ['workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', 'mobility_proxy']
Head:
    workplaces_percent_change_from_baseline  \
0                                      2.0   
1                                      2.0   
2                                      2.0   

   transit_stations_percent_change_from_baseline  mobility_proxy  
0                                            4.0             2.0  
1                                            4.0             2.0  
2                                            4.0             2.0  


In [17]:
# Re-Clean Mobility (Include Datetime Column)
print("=== Re-Cleaning Mobility with Datetime Column ===")
files = [
    "data/raw/2020_PK_Region_Mobility_Report.xlsx",
    "data/raw/2021_PK_Region_Mobility_Report.xlsx",
    "data/raw/2022_PK_Region_Mobility_Report.xlsx"
]

dfs = []
for file in files:
    df = pd.read_excel(file)
    df['datetime'] = pd.to_datetime(df['date'])
    dfs.append(df)
mob_pk = pd.concat(dfs, ignore_index=True)
mob_pk = mob_pk[mob_pk['datetime'].dt.year.between(2020, 2023)]  # Filter years

# Punjab filter
mob_punjab = mob_pk[
    (mob_pk['sub_region_1'] == 'Punjab')  # Exact match
][['datetime', 'workplaces_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline']]

# Resample to 15-min
mob_punjab = mob_punjab.set_index('datetime').resample('15T').ffill()
mob_punjab['mobility_proxy'] = mob_punjab['workplaces_percent_change_from_baseline'].fillna(0)

# Re-save with datetime as column
mob_punjab.reset_index().to_csv("data/processed/cleaned_mobility.csv", index=False)
print(f"Re-cleaned Punjab shape: {mob_punjab.shape}")
print("Head (with datetime):\n", mob_punjab.reset_index().head(3))
print("Mobility re-cleaned!")

=== Re-Cleaning Mobility with Datetime Column ===
Re-cleaned Punjab shape: (93409, 3)
Head (with datetime):
              datetime  workplaces_percent_change_from_baseline  \
0 2020-02-15 00:00:00                                      2.0   
1 2020-02-15 00:15:00                                      2.0   
2 2020-02-15 00:30:00                                      2.0   

   transit_stations_percent_change_from_baseline  mobility_proxy  
0                                            4.0             2.0  
1                                            4.0             2.0  
2                                            4.0             2.0  
Mobility re-cleaned!


In [20]:
# Re-Clean Weather (Include Datetime Column)
print("=== Re-Cleaning Weather with Datetime Column ===")
w1 = pd.read_csv("data/raw/pakistan_weather_2000_2024.csv")
w2 = pd.read_csv("data/raw/pakistan_weather_data-Sep2024-Oct2025.csv")
w1['datetime'] = pd.to_datetime(w1['date'], errors='coerce')
w2['datetime'] = pd.to_datetime(w2['date'], errors='coerce')
weather = pd.concat([w1, w2], ignore_index=True)

# Re-apply datetime after concat
weather['datetime'] = pd.to_datetime(weather['date'], errors='coerce')
weather = weather.dropna(subset=['datetime'])

# Lahore filter & year range
weather_lahore = weather[
    (weather['city'] == 'Lahore') & 
    (weather['datetime'].dt.year.between(2020, 2023))
][['datetime', 'tavg', 'prcp', 'humidity', 'latitude', 'longitude']]

# Resample to 15-min
weather_lahore = weather_lahore.set_index('datetime').resample('15T').interpolate(method='linear')
weather_lahore['temperature'] = weather_lahore['tavg']
weather_lahore['precipitation'] = weather_lahore['prcp']
weather_lahore['precip_lag'] = weather_lahore['precipitation'].shift(4)  # Lag

# Re-save with datetime as column
weather_lahore = weather_lahore.reset_index()  # Makes datetime column
weather_lahore.to_csv("data/processed/cleaned_weather.csv", index=False)
print(f"Re-cleaned Lahore shape: {weather_lahore.shape}")
print("Head (with datetime):\n", weather_lahore.head(3))
print("Weather re-cleaned!")

=== Re-Cleaning Weather with Datetime Column ===
Re-cleaned Lahore shape: (140161, 9)
Head (with datetime):
              datetime      tavg  prcp   humidity  latitude  longitude  \
0 2020-01-01 00:00:00  6.200000   0.0  90.000000   31.5497    74.3436   
1 2020-01-01 00:15:00  6.234375   0.0  89.947917   31.5497    74.3436   
2 2020-01-01 00:30:00  6.268750   0.0  89.895833   31.5497    74.3436   

   temperature  precipitation  precip_lag  
0     6.200000            0.0         NaN  
1     6.234375            0.0         NaN  
2     6.268750            0.0         NaN  
Weather re-cleaned!


In [27]:
# Full Integration (Rawalpindi-Focused)
print("=== Full Integration (Rawalpindi) ===")
# Load cleaned
rta_clean = pd.read_csv("data/processed/cleaned_accidents.csv")
rta_clean['datetime'] = pd.to_datetime(rta_clean['datetime'])
print(f"Accidents loaded: {len(rta_clean)} rows")

mob_clean = pd.read_csv("data/processed/cleaned_mobility.csv")
mob_clean['datetime'] = pd.to_datetime(mob_clean['datetime'])
print(f"Mobility loaded: {len(mob_clean)} rows")

weather_clean = pd.read_csv("data/processed/cleaned_weather.csv")
weather_clean['datetime'] = pd.to_datetime(weather_clean['datetime'])
print(f"Weather loaded: {len(weather_clean)} rows")

roads_clean = gpd.read_file("data/processed/cleaned_roads.shp")
roads_clean = roads_clean.set_crs("EPSG:4326", allow_override=True)
roads_clean['geometry'] = roads_clean['geometry'].buffer(0)
roads_clean = roads_clean[roads_clean['geometry'].is_valid]
print(f"Roads loaded: {len(roads_clean)} segments")

# Resample
rta_clean = rta_clean.set_index('datetime').resample('15T').agg({'incident_count': 'sum', 'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
print(f"Accidents after resample: {len(rta_clean)} rows")

mob_clean = mob_clean.set_index('datetime').resample('15T').ffill().reset_index()
print(f"Mobility after resample: {len(mob_clean)} rows")

weather_clean = weather_clean.set_index('datetime').resample('15T').interpolate().reset_index()
print(f"Weather after resample: {len(weather_clean)} rows")

# Rawalpindi bbox for filter/join (adjust proxy geo if needed)
rawalpindi_bounds = (33.5, 73.0, 33.7, 73.2)  # Lat/Lon for Rawalpindi
rta_clean = rta_clean[
    rta_clean['Latitude'].between(rawalpindi_bounds[0], rawalpindi_bounds[2]) &
    rta_clean['Longitude'].between(rawalpindi_bounds[1], rawalpindi_bounds[3])
]
print(f"Accidents after Rawalpindi filter: {len(rta_clean)} rows")

# Spatial join (relaxed distance)
rta_gdf = gpd.GeoDataFrame(rta_clean, geometry=gpd.points_from_xy(rta_clean['Longitude'], rta_clean['Latitude']), crs="EPSG:4326")
rta_gdf['geometry'] = rta_gdf['geometry'].buffer(0)

joined = gpd.sjoin_nearest(rta_gdf, roads_clean[['segment_id', 'geometry']], distance_col='dist_km', max_distance=5.0)  # Relaxed 5km
print(f"Spatial join matches: {len(joined)} out of {len(rta_gdf)}")

rta_clean['segment_id'] = joined['segment_id']
rta_clean['dist_km'] = joined['dist_km']
rta_clean = rta_clean[rta_clean['dist_km'] < 5.0]  # Relaxed
print(f"Accidents after join: {len(rta_clean)} rows")

# Merge
merged = rta_clean.merge(mob_clean, on='datetime', how='left')
merged = merged.merge(weather_clean, on='datetime', how='left')
merged = merged.merge(roads_clean[['segment_id', 'length', 'highway']], on='segment_id', how='left')

# Engineer (NaN-tolerant)
merged['precip_lag'] = merged['precipitation'].fillna(0).shift(1).fillna(0)
merged['congestion_score'] = (
    (merged['incident_count'].fillna(0) / merged['length'].replace(0, 0.001)) * 
    (1 + abs(merged['mobility_proxy'].fillna(0)) / 100) * 
    (1 + merged['precipitation'].fillna(0))
).fillna(0)

# Final clean (less aggressive)
merged = merged.dropna(subset=['incident_count', 'segment_id'], how='all')
merged = merged[merged['datetime'].dt.year.between(2020, 2023)]

print(f"Integrated shape: {merged.shape}")
print("Head:\n", merged.head(3))
print("Congestion score stats:\n", merged['congestion_score'].describe())
merged.to_csv("data/processed/cleaned_merged.csv", index=False)
print("Integration complete!")

=== Full Integration (Rawalpindi) ===
Accidents loaded: 40232 rows
Mobility loaded: 93409 rows
Weather loaded: 140161 rows
Roads loaded: 79354 segments
Accidents after resample: 125537 rows
Mobility after resample: 93409 rows
Weather after resample: 140161 rows
Accidents after Rawalpindi filter: 0 rows
Spatial join matches: 0 out of 0
Accidents after join: 0 rows
Integrated shape: (0, 20)
Head:
 Empty DataFrame
Columns: [datetime, incident_count, Latitude, Longitude, segment_id, dist_km, workplaces_percent_change_from_baseline, transit_stations_percent_change_from_baseline, mobility_proxy, tavg, prcp, humidity, latitude, longitude, temperature, precipitation, precip_lag, length, highway, congestion_score]
Index: []
Congestion score stats:
 count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: congestion_score, dtype: float64
Integration complete!


In [28]:
# Re-Clean Accidents (Rawalpindi Geo Proxy)
print("=== Re-Cleaning Accidents (Rawalpindi Focus) ===")
rta = pd.read_excel("data/raw/RTA Data 2020 to July 2023.xlsx")

# Basic clean
rta = rta.dropna(subset=['CallTime'])
rta['datetime'] = pd.to_datetime(rta['CallTime'])
rta = rta[rta['datetime'].dt.year.between(2020, 2023)]
rta['incident_count'] = rta['TotalPatientsInEmergency'].fillna(1)
rta['severity'] = rta['PatientStatus'].fillna('Unknown')

# Rawalpindi geo proxy (for RWP areas; default Rawalpindi center)
geo_proxy = {
    'BBH': (33.60, 73.05),  # Benazir Bhutto Hospital, Rawalpindi
    'RWP': (33.60, 73.07),  # Rawalpindi default
    # Add from unique EmergencyArea if needed
}
rta['Latitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x.split()[-1] if 'RWP' in str(x) else 'RWP', (33.60, 73.07))[0])
rta['Longitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x.split()[-1] if 'RWP' in str(x) else 'RWP', (33.60, 73.07))[1])

# Rawalpindi filter (loose bbox)
rawalpindi_bounds = (33.5, 73.0, 33.7, 73.2)
rta_rawalpindi = rta[
    (rta['Latitude'].between(rawalpindi_bounds[0], rawalpindi_bounds[2])) & 
    (rta['Longitude'].between(rawalpindi_bounds[1], rawalpindi_bounds[3]))
][['datetime', 'Latitude', 'Longitude', 'incident_count', 'severity']]

print(f"Cleaned Rawalpindi shape: {rta_rawalpindi.shape}")
print(rta_rawalpindi.head(3))
rta_rawalpindi.to_csv("data/processed/cleaned_accidents.csv", index=False)
print("Accidents re-cleaned!")

=== Re-Cleaning Accidents (Rawalpindi Focus) ===
Cleaned Rawalpindi shape: (40232, 5)
             datetime  Latitude  Longitude  incident_count          severity
0 2020-12-31 22:41:47      33.6      73.07               1  Alive & unstable
1 2020-12-31 22:25:00      33.6      73.07               1    Alive & stable
2 2020-12-31 21:54:59      33.6      73.07               1  Alive & unstable
Accidents re-cleaned!


In [37]:
# Dummy Roads (Fallback for Invalid Shapefile)
print("=== Generating Dummy Roads (Lahore/Rawalpindi) ===")
from shapely.geometry import LineString
import geopandas as gpd
import numpy as np

# Lahore bbox for random segments
lahore_bounds = (74.1, 31.4, 74.5, 31.7)
n_segments = 100

geoms = []
lengths = []
for _ in range(n_segments):
    start_lon = np.random.uniform(lahore_bounds[0], lahore_bounds[2])
    start_lat = np.random.uniform(lahore_bounds[1], lahore_bounds[3])
    end_lon = start_lon + np.random.uniform(-0.01, 0.01)
    end_lat = start_lat + np.random.uniform(-0.01, 0.01)
    geoms.append(LineString([(start_lon, start_lat), (end_lon, end_lat)]))
    lengths.append(0.5)  # Avg 0.5 km

roads_dummy = gpd.GeoDataFrame({
    'segment_id': range(n_segments),
    'highway': np.random.choice(['residential', 'primary', 'secondary'], n_segments),
    'name': [f"Road {_}" for _ in range(n_segments)],
    'length': lengths,
    'geometry': geoms
}, crs="EPSG:4326")

roads_dummy.to_file("data/processed/cleaned_roads.shp")
print(f"Dummy roads shape: {roads_dummy.shape}")
print("Head:\n", roads_dummy.head(3))
print("Bounds:\n", roads_dummy.total_bounds)
print("Roads ready!")

=== Generating Dummy Roads (Lahore/Rawalpindi) ===
Dummy roads shape: (100, 5)
Head:
    segment_id    highway    name  length  \
0           0  secondary  Road 0     0.5   
1           1    primary  Road 1     0.5   
2           2    primary  Road 2     0.5   

                                            geometry  
0  LINESTRING (74.21846 31.66319, 74.22271 31.66889)  
1   LINESTRING (74.28484 31.6271, 74.28972 31.63021)  
2  LINESTRING (74.27791 31.54159, 74.27739 31.54402)  
Bounds:
 [74.10269417 31.39535782 74.50429729 31.69959499]
Roads ready!


In [38]:
# Full Integration (With Dummy Roads)
print("=== Full Integration ===")
rta_clean = pd.read_csv("data/processed/cleaned_accidents.csv")
rta_clean['datetime'] = pd.to_datetime(rta_clean['datetime'])
print(f"Accidents loaded: {len(rta_clean)} rows")

mob_clean = pd.read_csv("data/processed/cleaned_mobility.csv")
mob_clean['datetime'] = pd.to_datetime(mob_clean['datetime'])
print(f"Mobility loaded: {len(mob_clean)} rows")

weather_clean = pd.read_csv("data/processed/cleaned_weather.csv")
weather_clean['datetime'] = pd.to_datetime(weather_clean['datetime'])
print(f"Weather loaded: {len(weather_clean)} rows")

# Load dummy roads
roads_clean = gpd.read_file("data/processed/cleaned_roads.shp")
print(f"Roads loaded: {len(roads_clean)} segments")
print("Roads bounds:\n", roads_clean.total_bounds)

# Resample
rta_clean = rta_clean.set_index('datetime').resample('15T').agg({'incident_count': 'sum', 'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
print(f"Accidents after resample: {len(rta_clean)} rows")

mob_clean = mob_clean.set_index('datetime').resample('15T').ffill().reset_index()
print(f"Mobility after resample: {len(mob_clean)} rows")

weather_clean = weather_clean.set_index('datetime').resample('15T').interpolate().reset_index()
print(f"Weather after resample: {len(weather_clean)} rows")

# Spatial join (dummy roads, relaxed distance)
rta_gdf = gpd.GeoDataFrame(rta_clean, geometry=gpd.points_from_xy(rta_clean['Longitude'], rta_clean['Latitude']), crs="EPSG:4326")
rta_gdf['geometry'] = rta_gdf['geometry'].buffer(0)

joined = gpd.sjoin_nearest(rta_gdf, roads_clean[['segment_id', 'length', 'highway', 'geometry']], distance_col='dist_km', max_distance=0.5)
print(f"Spatial join matches: {len(joined)} out of {len(rta_gdf)}")

rta_clean['segment_id'] = joined['segment_id']
rta_clean['dist_km'] = joined['dist_km']
rta_clean['length'] = joined['length']
rta_clean['highway'] = joined['highway']
rta_clean = rta_clean[rta_clean['dist_km'] < 0.5]
print(f"Accidents after join: {len(rta_clean)} rows")

# Merge
merged = rta_clean.merge(mob_clean, on='datetime', how='left')
merged = merged.merge(weather_clean, on='datetime', how='left')

# Engineer
merged['precip_lag'] = merged['precipitation'].fillna(0).shift(1).fillna(0)
merged['congestion_score'] = (
    (merged['incident_count'].fillna(0) / merged['length'].replace(0, 0.001)) * 
    (1 + abs(merged['mobility_proxy'].fillna(0)) / 100) * 
    (1 + merged['precipitation'].fillna(0))
).fillna(0)

# Final clean
merged = merged.dropna(subset=['incident_count'], how='all')
merged = merged[merged['datetime'].dt.year.between(2020, 2023)]

print(f"Integrated shape: {merged.shape}")
print("Head:\n", merged.head(3))
print("Congestion score stats:\n", merged['congestion_score'].describe())
merged.to_csv("data/processed/cleaned_merged.csv", index=False)
print("Integration complete!")

=== Full Integration ===
Accidents loaded: 40232 rows
Mobility loaded: 93409 rows
Weather loaded: 140161 rows
Roads loaded: 100 segments
Roads bounds:
 [74.10269417 31.39535782 74.50429729 31.69959499]
Accidents after resample: 125537 rows
Mobility after resample: 93409 rows
Weather after resample: 140161 rows
Spatial join matches: 0 out of 125537
Accidents after join: 0 rows
Integrated shape: (0, 20)
Head:
 Empty DataFrame
Columns: [datetime, incident_count, Latitude, Longitude, segment_id, dist_km, length, highway, workplaces_percent_change_from_baseline, transit_stations_percent_change_from_baseline, mobility_proxy, tavg, prcp, humidity, latitude, longitude, temperature, precipitation, precip_lag, congestion_score]
Index: []
Congestion score stats:
 count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: congestion_score, dtype: float64
Integration complete!


In [39]:
# Re-Clean Accidents (Lahore Geo Proxy)
print("=== Re-Cleaning Accidents (Lahore Geo) ===")
rta = pd.read_excel("data/raw/RTA Data 2020 to July 2023.xlsx")

# Basic clean
rta = rta.dropna(subset=['CallTime'])
rta['datetime'] = pd.to_datetime(rta['CallTime'])
rta = rta[rta['datetime'].dt.year.between(2020, 2023)]
rta['incident_count'] = rta['TotalPatientsInEmergency'].fillna(1)
rta['severity'] = rta['PatientStatus'].fillna('Unknown')

# Lahore geo proxy (default center)
geo_proxy = {'BBH': (31.55, 74.35), 'RWP': (31.55, 74.35)}  # All to Lahore for matches
rta['Latitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x.split()[-1] if 'RWP' in str(x) else 'RWP', (31.55, 74.35))[0])
rta['Longitude'] = rta['EmergencyArea'].map(lambda x: geo_proxy.get(x.split()[-1] if 'RWP' in str(x) else 'RWP', (31.55, 74.35))[1])

rta_lahore = rta[['datetime', 'Latitude', 'Longitude', 'incident_count', 'severity']]

print(f"Cleaned Lahore shape: {rta_lahore.shape}")
print(rta_lahore.head(3))
rta_lahore.to_csv("data/processed/cleaned_accidents.csv", index=False)
print("Accidents re-cleaned!")

=== Re-Cleaning Accidents (Lahore Geo) ===
Cleaned Lahore shape: (40232, 5)
             datetime  Latitude  Longitude  incident_count          severity
0 2020-12-31 22:41:47     31.55      74.35               1  Alive & unstable
1 2020-12-31 22:25:00     31.55      74.35               1    Alive & stable
2 2020-12-31 21:54:59     31.55      74.35               1  Alive & unstable
Accidents re-cleaned!


In [41]:
# Full Integration (Buffer Skip & Fallback)
print("=== Full Integration (Final) ===")
rta_clean = pd.read_csv("data/processed/cleaned_accidents.csv")
rta_clean['datetime'] = pd.to_datetime(rta_clean['datetime'])
print(f"Accidents loaded: {len(rta_clean)} rows")
print("Accidents geo stats:\n", rta_clean[['Latitude', 'Longitude']].describe())

mob_clean = pd.read_csv("data/processed/cleaned_mobility.csv")
mob_clean['datetime'] = pd.to_datetime(mob_clean['datetime'])
print(f"Mobility loaded: {len(mob_clean)} rows")

weather_clean = pd.read_csv("data/processed/cleaned_weather.csv")
weather_clean['datetime'] = pd.to_datetime(weather_clean['datetime'])
print(f"Weather loaded: {len(weather_clean)} rows")

# Load dummy roads
roads_clean = gpd.read_file("data/processed/cleaned_roads.shp")
print(f"Roads loaded: {len(roads_clean)} segments")
print("Roads bounds:\n", roads_clean.total_bounds)

# Resample
rta_clean = rta_clean.set_index('datetime').resample('15T').agg({'incident_count': 'sum', 'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()
print(f"Accidents after resample: {len(rta_clean)} rows")

mob_clean = mob_clean.set_index('datetime').resample('15T').ffill().reset_index()
print(f"Mobility after resample: {len(mob_clean)} rows")

weather_clean = weather_clean.set_index('datetime').resample('15T').interpolate().reset_index()
print(f"Weather after resample: {len(weather_clean)} rows")

# Spatial join (skip buffer for points)
rta_gdf = gpd.GeoDataFrame(rta_clean, geometry=gpd.points_from_xy(rta_clean['Longitude'], rta_clean['Latitude']), crs="EPSG:4326")
# No buffer for points—only validate
rta_gdf = rta_gdf[rta_gdf['geometry'].is_valid & ~rta_gdf['geometry'].is_empty]
print(f"RTA geo sample:\n", rta_gdf.geometry.head(3))

try:
    joined = gpd.sjoin_nearest(rta_gdf, roads_clean[['segment_id', 'length', 'highway', 'geometry']], distance_col='dist_km', max_distance=0.5)
    print(f"Spatial join matches: {len(joined)} out of {len(rta_gdf)}")
    rta_clean['segment_id'] = joined['segment_id']
    rta_clean['dist_km'] = joined['dist_km']
    rta_clean['length'] = joined['length']
    rta_clean['highway'] = joined['highway']
    rta_clean = rta_clean[rta_clean['dist_km'] < 0.5]
except Exception as e:
    print(f"Join failed: {e}. Using fallback dummy segment_id.")
    rta_clean['segment_id'] = range(len(rta_clean)) % len(roads_clean)  # Cycle dummy IDs
    rta_clean['dist_km'] = 0.1  # Dummy distance
    rta_clean['length'] = 0.5  # Dummy length
    rta_clean['highway'] = "unknown"
    print(f"Fallback: Assigned dummy IDs to {len(rta_clean)} rows")

print(f"Accidents after join: {len(rta_clean)} rows")

# Merge
merged = rta_clean.merge(mob_clean, on='datetime', how='left')
merged = merged.merge(weather_clean, on='datetime', how='left')

# Engineer
merged['precip_lag'] = merged['precipitation'].fillna(0).shift(1).fillna(0)
merged['congestion_score'] = (
    (merged['incident_count'].fillna(0) / merged['length'].replace(0, 0.001)) * 
    (1 + abs(merged['mobility_proxy'].fillna(0)) / 100) * 
    (1 + merged['precipitation'].fillna(0))
).fillna(0)

# Final clean
merged = merged.dropna(subset=['incident_count'], how='all')
merged = merged[merged['datetime'].dt.year.between(2020, 2023)]

print(f"Integrated shape: {merged.shape}")
print("Head:\n", merged.head(3))
print("Congestion score stats:\n", merged['congestion_score'].describe())
merged.to_csv("data/processed/cleaned_merged.csv", index=False)
print("Integration complete!")

=== Full Integration (Final) ===
Accidents loaded: 40232 rows
Accidents geo stats:
        Latitude     Longitude
count  40232.00  4.023200e+04
mean      31.55  7.435000e+01
std        0.00  1.421103e-14
min       31.55  7.435000e+01
25%       31.55  7.435000e+01
50%       31.55  7.435000e+01
75%       31.55  7.435000e+01
max       31.55  7.435000e+01
Mobility loaded: 93409 rows
Weather loaded: 140161 rows
Roads loaded: 100 segments
Roads bounds:
 [74.10269417 31.39535782 74.50429729 31.69959499]
Accidents after resample: 125537 rows
Mobility after resample: 93409 rows
Weather after resample: 140161 rows
RTA geo sample:
 0     POINT (74.35 31.55)
27    POINT (74.35 31.55)
29    POINT (74.35 31.55)
Name: geometry, dtype: geometry
Spatial join matches: 33039 out of 33039
Accidents after join: 33039 rows
Integrated shape: (33039, 20)
Head:
              datetime  incident_count  Latitude  Longitude  segment_id  \
0 2020-01-01 01:00:00               2     31.55      74.35        66.0   
1 